In [1]:
import os
os.chdir('/rds/projects/j/jenseno-visual-search-rft/AET NN')


In [2]:
import optuna
import torch
from torch import nn

import numpy as np
import aet_net
from itertools import combinations

import matplotlib.pyplot as plt

In [4]:
import pickle
with open('optuna_aet_trial.pkl','rb') as fp:
    optuna_aet_result = pickle.load(fp)
    print('optuna result')
    print(optuna_aet_result)

optuna result
[{'loss': 2.7408602237701416}, {'ETA_': 0.1110814367272019, 'BETA_': 0.0076329758600648286, 'P_': 0.0006413053701833462, 'slope_sig': 4}]


In [15]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# model parameters

nn_dim_ = [28,68,3]   # [quadrant size, number of hidden nodes, number of output nodes]
eta_ = optuna_aet_result[1]['ETA_']          # learning rate
mini_sz_ = 1          # mini batch size (1 = use SGD)
num_epo_ = 80
kl_reg_ = [optuna_aet_result[1]['BETA_'], optuna_aet_result[1]['P_']]  # identified with optuna
sig_param = [1, 0] # sigmoid slope and shift in x direction

# loss function & final layer activation (for binary crossentropy use sigmoid)
lossfun = [nn.MSELoss(), nn.Softmax(dim=-1)]

params = nn_dim_,eta_,mini_sz_,num_epo_,kl_reg_,sig_param

# initialize model and weights
model = aet_net.net(params,lossfun)
model = aet_net.init_params(model,weight_init='uni')
optimizer = torch.optim.SGD(model.parameters(),lr=eta_)

model.to(DEVICE)
loss_hist = model.train(optimizer,noise=False,print_loss=False)

In [16]:
# dynamics indiviudal input
import aet_dyn

plt.rcParams["figure.figsize"] = (10,5)
font = {'family' : 'Arial',
        'weight' : 'normal',
        'size'   : 16}
plt.rc('font',**font)


alpha_params = [10,3]
dyn_params = [0.01,0.05,4,0.05,0,1]   # [tau_h, tau_r, r_scale factor, T, h start, R start]
_io = 0

timevec = np.linspace(0,0.6,600)

Z_t,H_t, R_t, O_t  = aet_dyn.euler_dyn(model,data[0], dyn_params, timevec, alpha_params,DEVICE,inp_on=_io)

fig,ax = plt.subplots(3,1)
ax[2].plot(O_t.T.cpu().detach().numpy())
ax[2].legend(('A(t)','E(t)','T(t)','Z(t)'),loc='lower right')
ax[2].set_title('output')

ax[1].plot(R_t.T.cpu().detach().numpy())
ax[1].set_title('relaxation')
ax[0].plot(H_t.T.cpu().detach().numpy())
ax[0].set_title('hidden acti')

if alpha_params[1]:
    alpha_inh = 0.5*np.sin(2*np.pi*timevec*10)+0.5
    ax[2].plot(alpha_inh,'k',linewidth=0.5,linestyle='-.')
    ax[0].plot(alpha_inh,'k',linewidth=1,linestyle='-.')
fig.tight_layout()

fig.savefig('dyna_alpha_A.png')


NameError: name 'data' is not defined